## Get text embeddings

- update `sepsis_removed_0.pkl`

    - update `data` with `emb` column
    - concat text embeddings per instance to generate direct input to model


## Hardware check

In [1]:
# gpu check
! nvidia-smi

Mon Jun 12 16:55:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   43C    P0    40W / 300W |      9MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:B2:00.0 Off |                    0 |
| N/A   

In [2]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

In [3]:
! python --version

Python 3.9.7


## Environment Prep

In [4]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis/

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [5]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
from keras.utils import pad_sequences
# from tensorflow.keras.models import Model
from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
# tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
from transformers import AutoTokenizer, pipeline, AutoModel
import resources.smart_cond as sc
# from google.colab import files

2023-06-12 16:56:00.501232: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 16:56:00.551140: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 16:56:01.606363: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Version Check

In [12]:
import tensorflow as tf
print(tf. __version__)

pickle.format_version

2.12.0


'4.0'

## Train/Val

In [7]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
embs = pkl[5]
del pkl

In [8]:
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date: [**2119-5-4**] D...,noteevents,1.000000,1.000000
1,18407,28.016667,Text,Admission Date: [**2112-12-8**] ...,noteevents,1.000000,1.000000
2,40300,155.166667,Text,Admission Date: [**2194-7-18**] ...,noteevents,1.000000,1.000000
3,23747,52.383333,Text,Admission Date: [**2194-1-7**] D...,noteevents,1.000000,1.000000
4,2357,73.133333,Text,Admission Date: [**2186-6-7**] Discharge ...,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [9]:
text_data = data[data['variable'] == 'Text']
text_data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date: [**2119-5-4**] D...,noteevents,1.0,1.0
1,18407,28.016667,Text,Admission Date: [**2112-12-8**] ...,noteevents,1.0,1.0
2,40300,155.166667,Text,Admission Date: [**2194-7-18**] ...,noteevents,1.0,1.0
3,23747,52.383333,Text,Admission Date: [**2194-1-7**] D...,noteevents,1.0,1.0
4,2357,73.133333,Text,Admission Date: [**2186-6-7**] Discharge ...,noteevents,1.0,1.0
...,...,...,...,...,...,...,...
1407425,31015,99.400000,Text,NPN\n\n\n#1 Infant remains in RA with O2 sats...,noteevents,1.0,1.0
1407426,31015,105.783333,Text,"Neonatology\nDOL #5, CGA 36 weeks. \n\nCVR: Co...",noteevents,1.0,1.0
1407427,31015,112.650000,Text,Family Meeting Note\nFamily meeting held with ...,noteevents,1.0,1.0
1407428,31015,114.033333,Text,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...,noteevents,1.0,1.0


In [10]:
embs_list = []

for emb in embs:
    embs_list.append(emb)

text_data['value'] = embs_list
text_data

/scratch/slurm_tmpdir/job_22329728/ipykernel_171960/3937955675.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data['value'] = embs_list


,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,"[0.18356061, -0.016009603, -0.22388706, 0.0302...",noteevents,1.0,1.0
1,18407,28.016667,Text,"[0.060869485, 0.04175006, -0.10215783, 0.09201...",noteevents,1.0,1.0
2,40300,155.166667,Text,"[0.0680666, -0.00074727647, -0.19209613, 0.065...",noteevents,1.0,1.0
3,23747,52.383333,Text,"[-0.05982084, 0.015669934, -0.07559425, 0.0844...",noteevents,1.0,1.0
4,2357,73.133333,Text,"[-0.0005084318, -0.023331445, -0.20704384, 0.0...",noteevents,1.0,1.0
...,...,...,...,...,...,...,...
1407425,31015,99.400000,Text,"[-0.07868559, -0.11163999, -0.30744517, 0.0592...",noteevents,1.0,1.0
1407426,31015,105.783333,Text,"[-0.03863062, 0.041172553, -0.14782721, 0.1501...",noteevents,1.0,1.0
1407427,31015,112.650000,Text,"[0.11279309, -0.079024784, -0.20364788, 0.0287...",noteevents,1.0,1.0
1407428,31015,114.033333,Text,"[-0.014392208, -0.079443246, -0.17264834, 0.03...",noteevents,1.0,1.0


In [11]:
physio_data = data[data['variable'] != 'Text']
physio_data

,ts_ind,hour,variable,value,TABLE,mean,std
1407430,0,0.000000,Age,66.0,N/A,64.053647,56.625699
1407431,0,0.000000,Gender,1.0,N/A,0.438951,0.496263
1407432,0,0.033333,DBP,-0.517967,chart,59.766756,14.994705
1407433,0,0.033333,GCS_eye,0.679313,chart,3.274060,1.068640
1407434,0,0.033333,GCS_motor,0.515191,chart,5.271144,1.414728
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [12]:
del data
data = text_data.append(physio_data, ignore_index=False)

/scratch/slurm_tmpdir/job_22329728/ipykernel_171960/2221810867.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = text_data.append(physio_data, ignore_index=False)


In [13]:
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,"[0.18356061, -0.016009603, -0.22388706, 0.0302...",noteevents,1.000000,1.000000
1,18407,28.016667,Text,"[0.060869485, 0.04175006, -0.10215783, 0.09201...",noteevents,1.000000,1.000000
2,40300,155.166667,Text,"[0.0680666, -0.00074727647, -0.19209613, 0.065...",noteevents,1.000000,1.000000
3,23747,52.383333,Text,"[-0.05982084, 0.015669934, -0.07559425, 0.0844...",noteevents,1.000000,1.000000
4,2357,73.133333,Text,"[-0.0005084318, -0.023331445, -0.20704384, 0.0...",noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [14]:
pred_window = 2 # hours
obs_windows = range(20, 124, 4)

# Remove test patients.
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(test_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(test_sub)]
data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1

# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
fore_texts_ip = []
fore_inds = []
def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask
def pad(x):
    return x+[0]*(fore_max_len-len(x))
for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)    
    obs_data = data.loc[(data.hour<w)&(data.hour>=w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind':list, 'hour':list, 'value':list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_inds.append(np.array(list(obs_data.ts_ind)))
    
    matrix = list(obs_data.value)
    obs_strings = []
    for l in matrix:
        string_list = []
        for value in l:
            if not np.isscalar(value):
            # if isinstance(value, str):
                string_list.append(value)
        obs_strings.append(string_list)
    del matrix
    fore_texts_ip.append(np.array(obs_strings)) 
del data

  0%|          | 0/26 [00:00<?, ?it/s]/scratch/slurm_tmpdir/job_22329728/ipykernel_171960/2820743741.py:70: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fore_texts_ip.append(np.array(obs_strings))
  4%|▍         | 1/26 [00:20<08:32, 20.48s/it]/scratch/slurm_tmpdir/job_22329728/ipykernel_171960/2820743741.py:70: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fore_texts_ip.append(np.array(obs_strings))
  8%|▊         | 2/26 [00:41<08:19, 20.81s/it]/scratch/slurm_tmpdir/job_22329728/ipykernel_171960/2820743741.py:70: VisibleDeprecationWarning: Creating 

In [15]:
fore_texts_ip = np.concatenate(fore_texts_ip, axis=0)
fore_inds = np.concatenate(fore_inds, axis=0)
# Get train and valid ts_ind for forecast task.
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
rem_sub = oc.loc[~oc.SUBJECT_ID.isin(np.concatenate((train_ind, valid_ind)))].SUBJECT_ID.unique()
bp = int(0.8*len(rem_sub))
train_sub = np.concatenate((train_sub, rem_sub[:bp]))
valid_sub = np.concatenate((valid_sub, rem_sub[bp:]))
train_ind = oc.loc[oc.SUBJECT_ID.isin(train_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
valid_ind = oc.loc[oc.SUBJECT_ID.isin(valid_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
# Generate 3 sets of inputs and outputs.
train_ind = np.argwhere(np.in1d(fore_inds, train_ind)).flatten()
valid_ind = np.argwhere(np.in1d(fore_inds, valid_ind)).flatten()
fore_train_ip = [ip[train_ind] for ip in [fore_texts_ip]]
fore_valid_ip = [ip[valid_ind] for ip in [fore_texts_ip]]
del fore_texts_ip

fore_train_text_ip = fore_train_ip[0]
fore_valid_text_ip = fore_valid_ip[0]
del fore_train_ip, fore_valid_ip

In [16]:
fore_train_concat_text_ip = []
# concat train texts per instance
for text in tqdm(fore_train_text_ip):
    if text == []:
        fore_train_concat_text_ip.append(np.array([0.0]))
    else: 
        concat_text = np.concatenate(text)
        fore_train_concat_text_ip.append(concat_text)
# train_embs = np.array(fore_train_concat_text_ip)        

100%|██████████| 449482/449482 [00:04<00:00, 97863.15it/s] 


In [17]:
fore_valid_concat_text_ip = []
# concat train texts per instance
for text in tqdm(fore_valid_text_ip):
    if text == []:
        fore_valid_concat_text_ip.append(np.array([0.0]))
    else: 
        concat_text = np.concatenate(text)
        fore_valid_concat_text_ip.append(concat_text)
# train_embs = np.array(fore_train_concat_text_ip)        

100%|██████████| 136823/136823 [00:01<00:00, 92656.21it/s]


In [18]:
# padding
max_len = 33792
# padding
padded_train = pad_sequences(fore_train_concat_text_ip, maxlen=max_len, dtype='float32', padding='post')
padded_train.shape

(449482, 33792)

In [19]:
# padding
padded_valid = pad_sequences(fore_valid_concat_text_ip, maxlen=max_len, dtype='float32', padding='post')
padded_valid.shape

(136823, 33792)

In [20]:
# dump to pkl, update sepsis_removed_0.pkl
pickle.dump([padded_train, padded_valid], open('Data/text_emb_input_train_val_0.pkl','wb'))

## Test

In [ ]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
embs = pkl[5]
del pkl
text_data = data[data['variable'] == 'Text']
embs_list = []
for emb in embs:
    embs_list.append(emb)
text_data['value'] = embs_list
physio_data = data[data['variable'] != 'Text']
del data
data = text_data.append(physio_data, ignore_index=False)

/scratch/slurm_tmpdir/job_22329728/ipykernel_202939/2618147854.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data['value'] = embs_list
/scratch/slurm_tmpdir/job_22329728/ipykernel_202939/2618147854.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = text_data.append(physio_data, ignore_index=False)


In [ ]:
pred_window = 2  # hours
obs_windows = range(20, 124, 4)

# Remove train, val patients
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(train_sub)]
data = data.loc[~data.SUBJECT_ID.isin(valid_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(train_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(valid_sub)]

data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]


def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d


static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1

# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']
            ].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
fore_texts_ip = []
fore_inds = []


def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask


def pad(x):
    return x+[0]*(fore_max_len-len(x))


for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour >= w) & (data.hour <= w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg(
        {'value': 'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg(
        {'vind_value': list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)
    obs_data = data.loc[(data.hour < w) & (data.hour >= w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg(
        {'vind': list, 'hour': list, 'value': list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_inds.append(np.array(list(obs_data.ts_ind)))

    matrix = list(obs_data.value)
    obs_strings = []
    for l in matrix:
        string_list = []
        for value in l:
            if not np.isscalar(value):
            # if isinstance(value, str):
                string_list.append(value)
        obs_strings.append(string_list)
    del matrix
    fore_texts_ip.append(np.array(obs_strings))
del data

fore_texts_ip = np.concatenate(fore_texts_ip, axis=0)

  0%|          | 0/26 [00:00<?, ?it/s]/scratch/slurm_tmpdir/job_22329728/ipykernel_202939/2484361121.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fore_texts_ip.append(np.array(obs_strings))
  4%|▍         | 1/26 [00:04<02:03,  4.95s/it]/scratch/slurm_tmpdir/job_22329728/ipykernel_202939/2484361121.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fore_texts_ip.append(np.array(obs_strings))
  8%|▊         | 2/26 [00:10<02:01,  5.05s/it]/scratch/slurm_tmpdir/job_22329728/ipykernel_202939/2484361121.py:88: VisibleDeprecationWarning: Creating 

In [ ]:
fore_test_concat_text_ip = []
# concat train texts per instance
for text in tqdm(fore_texts_ip):
    if text == []:
        fore_test_concat_text_ip.append(np.array([0.0]))
    else: 
        concat_text = np.concatenate(text)
        fore_test_concat_text_ip.append(concat_text)
# train_embs = np.array(fore_train_concat_text_ip)        

100%|██████████| 131920/131920 [00:00<00:00, 136868.71it/s]


In [28]:
# # obtain max_len, max_len = 33792
# max_length = max([len(max(fore_test_concat_text_ip, key=len)), len(max(fore_train_concat_text_ip, key=len)), len(max(fore_valid_concat_text_ip, key=len))])

33792

In [29]:
# # padding
# padded_train = pad_sequences(fore_train_concat_text_ip, maxlen=max_length, dtype='float32', padding='post')
# padded_train.shape

(449482, 33792)

In [30]:
# # padding
# padded_valid = pad_sequences(fore_valid_concat_text_ip, maxlen=max_length, dtype='float32', padding='post')
# padded_valid.shape

(136823, 33792)

In [10]:
# padding
max_len = 33792
padded_test = pad_sequences(fore_test_concat_text_ip, maxlen=max_len, dtype='float32', padding='post')
padded_test.shape

(131920, 33792)

In [32]:
# data_path = 'Data/sepsis_removed_0.pkl'
# pkl = pickle.load(open(data_path, 'rb'))
# data = pkl[0]
# oc = pkl[1]
# train_ind = pkl[2]
# valid_ind = pkl[3]
# test_ind = pkl[4]
# embs = pkl[5]
# del pkl
# text_data = data[data['variable'] == 'Text']
# embs_list = []
# for emb in embs:
#     embs_list.append(emb)
# text_data['value'] = embs_list
# physio_data = data[data['variable'] != 'Text']
# data_more = text_data.append(physio_data, ignore_index=False)

In [11]:
# dump to pkl, update sepsis_removed_0.pkl
pickle.dump([padded_test], open('Data/text_emb_input_test_0.pkl','wb'))